In [1]:
import os
import string
import math
from time import time
import editdistance
import pandas as pd 
import numpy as np
from numpy.random import randint
from linkage.derived import EntityFinder
from linkage.core.classifier import EntityPredicate, ScorePredicate, comparators, RulesBasedClassifier
from linkage.core.clustering.disjoint_set_clusters import get_clusters
import scipy 

BUILD_DIR = "/build/paper_benchmarks/music"

## Benchmark Linkage on MusicBrainz dataset

In this exercise, we compute the accuracy of linkage on a public dataset containing 20k records from the [MusicBrainz](https://musicbrainz.org/) database 


### Music Dataset

The dataset, along with some papers published on it, can be found [here](https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution)

5 sources

Here is a description of the data:

- TID: a unique record's id (in the complete dataset). <br/>
- CID: cluster id (records having the same CID are duplicate) <br/>
- CTID: a unique id within a cluster (if two records belong to the same cluster they will have the same CID but different CTIDs). These ids (CTID) start with 1 and grow until cluster size. <br/>
- SourceID: identifies to which source a record belongs (there are five sources). The sources are deduplicated. <br/>
- Id: the original id from the source. Each source has its own Id-Format. Uniqueness is not guaranteed!! (can be ignored). <br/>
- number: track or song number in the album. <br/>
- length: the length of the track. <br/>
- artist: the interpreter (artist or band) of the track. <br/>
- year: date of publication. <br/>
- language: language of the track. <br/>


### Read data 

In [2]:
def fetch_data():
    df = pd.read_csv("/build/musicbrainz-20k.csv")  # downloaded from the link mentioned above
    df['TID'] = df.TID.astype(str)
    cluster_size = df.groupby("CID").TID.count()
    df['CID_size'] = df.CID.map(cluster_size)
    print("Shape: %d X %d" % df.shape)
    print("Number of clusters: %d" % df.CID.nunique())
    return df

df = fetch_data()
df.sort_values("CID").head(5)  # CID identifies clusters of duplicates

Shape: 19375 X 13
Number of clusters: 10000


,TID,CID,CTID,SourceID,id,number,title,length,artist,album,year,language,CID_size
0,1,1,1,2,MBox7368722-HH,9,Daniel Balavoine - L'enfant aux yeux d'Italie,219,NaN,De vous à elle en passant par moi,75,French,2
15183,15184,1,2,3,11854016MB-01,9,L'enfant aux yeux d'Italie - De vous à elle en...,3.663,Daniel Balavoine,NaN,'75,French,2
14721,14722,2,2,3,6183919MB-01,17,Mustard Gas - There and Back Again Lane,2.15,Action Painting!,NaN,'95,English,2
2,3,2,1,2,MBox38440522-HH,17,Action PAINTING! - Mustard Gas,129,NaN,There and Back Again Lane,95,English,2
3,4,3,1,5,4489993,10,Your Grace,unk.,Kathy Troccoli,Comfort,2005,English,1


### Initialize Linkage

Configure linkage to run on this dataset. 

We use a simple unsupervised classifier here, which just checks whether the matching score is above the given threshold to classify it as a match. It is called "unsupervised" because the algorithm is not learning patterns from labeled data. We just try a few thresholds and adjust them depending on the clusters we get. If the clusters are too big, we reduce the threshold; if they are too small, we increase it. We do not use true labels to identify thresholds.

In [3]:
def deduplicate(df, th, min_threshold: float = 0.2, num_best: int = 5):
    
    # Describe columns to be used
    column_info = {
        'TID': {'type': 'string', 'block': False, 'classify': False, 'primary_key': True},
        'title': {'type': 'string', 'block': True, 'classify': True},
        'artist': {'type': 'string', 'block': True, 'classify': True},
        'album': {'type': 'string', 'block': True, 'classify': True},
        'language': {'type': 'string', 'block': True, 'classify': True},  
        'number': {'type': 'string', 'block': True, 'classify': True},    
        'length': {'type': 'string', 'block': True, 'classify': True},    
        'year': {'type': 'string', 'block': True, 'classify': True},    
    }

    # Use a simple threshold-based "unsupervised" classifer. This classifer can be changed to add more 
    # "semi-supervised" rules, but for the scope of this paper, we are not reporting the benchmarks 
    # from a supervised classifer – since training a model on the true labels provided in the dataset is not a fair 
    # comparison against benchmarks from papers which do not use the true labels.
    rbc_classifier = RulesBasedClassifier([
        [
        ScorePredicate(threshold=th) 
        ]      
    ])

    # Initialize EntityFinder class, which allows us to run entity resolution with a simple configuration
    ef = EntityFinder(column_info, classifier=rbc_classifier, build_dir=BUILD_DIR, query_method="inverted_index") 
    # create index on the dataset for fast processing; use out-of-core to use low memory.
    ef.create_index(df, out_of_core=True)
    ef.save() 
    del ef
    
    # In order to make sure the save/load function works, we delete the index object and re-load it from disk
    ef = EntityFinder.Load(BUILD_DIR, mmap=True, classifier=rbc_classifier)  # seq index requires save and load  
    # We search for matching entities on the same dataframe. `search` returns a list of matches. 
    # Each entry in the list contains the original entity, and a list of matching entities with 
    # corresponding matching scores from the Indexer. These scores, by default, are the cosine similarities 
    # between the entities. A matching entities has scores above the threshold specified in the classifier
    matches = ef.search(df, min_threshold=min_threshold, num_best=num_best, chunk_size=500000)

    # Form clusters between matching entities
    clusters, scores = get_clusters(matches)

    # Add clustering information to the original dataframe, and we are done!
    df['cluster_number'] = clusters
    df['cluster_score'] = df.cluster_number.apply(lambda i: scores.loc[(i, 'cluster_score')])
    df['lowest_score'] = df.cluster_number.apply(lambda i: scores.loc[(i, 'lowest_score')])
    df['cluster_size'] = df.cluster_number.apply(lambda i: scores.loc[(i, 'size')])
    
    return df

### Setup Evaluation Code

In [4]:
def nCr(n, r):
    """ N Choose R
    """
    if n == 1:
        return 1
    return scipy.special.comb(n, r)

def compute_tp_pp(tdf, col):
    """
    Compute true positives and predicted positives for the given dataframe and the column in question.
    If there are five entities in a predicted cluster, the predicted positives are 5 Choose 2 = 10
    If the corresponding column `col`, which contains ground truth, has a cluster of 3 entities, and another 
    cluster of 2, the true positives are (3 Choose 2) + (2 choose 2) = 5
    So, in this example, there were 10 predicted links (PP), but only 5 true links (TP)
    This can be used for computing overall precision of the algorithm. 
    
    The definition can be reversed: we can also compute positives and true positives
    using this function, which would cumulatively give us the recall of the algoritm
    
    Clusters containing only one entity are not counted towards either metrics.
    """
    if len(tdf) == 1:  # Cluster contains one entry, ignore
        return pd.Series({'pps': 0, 'tps': 0})
    
    pps = nCr(len(tdf), 2)  # predicted positives are just N choose 2
    
    # True positives is the \sum_{i=1}^{n} (N_i Choose 2) where N_i is the number of elements in
    # each predicted cluster and there are n predicted clusters. We ignore clusters of size 1.
    vcs = tdf[col].value_counts()    
    tps = vcs[vcs > 1].apply(lambda x: nCr(x, 2)).sum()  
    
    return pd.Series({'pps': pps, 'tps': tps})
    
def compute_precision_recall(df):
    """ Compute precision recall metrics by 
        - calling `compute_tp_pp` for each predicted cluster (for precision) and 
          each true cluster (for recall) 
        - Then, compute sum(tps) / sum(pps)
    """
    df_precision = df.groupby("cluster_number").apply(lambda x: compute_tp_pp(x, 'CID'))
    df_recall = df.groupby("CID").apply(lambda x: compute_tp_pp(x, 'cluster_number'))
    precision = df_precision.tps.sum() / df_precision.pps.sum()
    recall = df_recall.tps.sum() / df_recall.pps.sum()
    
    f_measure =  2 * precision * recall / (precision + recall)
    num_clusters = df.cluster_number.nunique()    
    return {
        "precision": precision, 
        "recall": recall, 
        "f_measure": f_measure, 
        "num_clusters": num_clusters, 
        "precision_identified_links": df_precision.sum()['pps'],
        "precision_correctly_identified_links": df_precision.sum()['tps'],
        "recall_num_links": df_recall.sum()['pps'],
        "recall_correctly_identified_links": df_recall.sum()['tps']
    }

### Run Linkage and Evaluation

In [5]:
df_output = deduplicate(df.copy(deep=True), th=0.45, min_threshold=0.3, num_best=10)
metrics = compute_precision_recall(df_output)
metrics

2022-01-20 20:20:07,631 : INFO : Ingesting data into an OOC embedded database
2022-01-20 20:20:07,632 : INFO : Phone number columns: []
2022-01-20 20:20:07,633 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:20:07,634 : INFO : Address columns: []
2022-01-20 20:20:08,320 : INFO : Blocking 19375 records on columns ['title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:20:08,418 : INFO : Using out-of-core tokenizer 
2022-01-20 20:20:16,238 : INFO : TFIDF blocker created with 31991 tokens
2022-01-20 20:20:16,240 : INFO : Saving EntityFinder blocked model to /build/paper_benchmarks/music
2022-01-20 20:20:17,387 : INFO : Loading using memorymap: r
2022-01-20 20:20:17,498 : INFO : Set dummy = -1
2022-01-20 20:20:17,499 : INFO : Collecting dictionary stat
2022-01-20 20:20:17,516 : INFO : Actually creating the index
2022-01-20 20:20:17,558 : INFO : Sanity check
2022-01-20 20:20:17,570 : INFO : Phone n

{'precision': 0.8758509990590579,
 'recall': 0.9737846153846154,
 'f_measure': 0.9222251362298568,
 'num_clusters': 9828,
 'precision_identified_links': 18067.0,
 'precision_correctly_identified_links': 15824.0,
 'recall_num_links': 16250.0,
 'recall_correctly_identified_links': 15824.0}

The recall is pretty high but the precision is low. This is because a few clusters are too big; and since we compute N choose 2, the errors get squared. Just fixing a couple of rogue clusters improves the precision greatly.

To "fix" these large clusters, we do not use the true labels, since that would be cheating!. We use `cluster_score` returned by Linkage, which gives a measure of the "connectivity" of edges in a cluster. 

It is computed as: number of actual edges in the cluster /  maximum possible edges

maximum possible edges = 2 * (N choose 2)

For example, if a cluster has three entities A, B, C and the edges are  A -> B, C -> B, B ->C
then, the cluster score is `3 / (2*(3 Choose 2))`  = `3 / 6` = `0.5`

If the `cluster_score` for a given cluser is below the threshold (0.8 in this case), we send the cluster to match again with a higher threshold.
We call this approach "iterative connected components" (apologies if the name is already taken by a different algorithm in the literature)

### Break clusters with low cluster_score

In [6]:
MAX_NUM = 100000000  # Trick used to generate unique cluster numbers
TH_CLUSTER_BREAK = 0.8

def run(th):
    """
    Runs deduplication iteratively and breaks down clusters with low `cluster_score` by sending them 
    through LINKAGE with a higher threshold
    :param th: threshold to run the algorithm on
    """
    df = fetch_data()
    df.set_index("TID", inplace=True, drop=False)
    for th in np.arange(th, th + 0.31, 0.1):  # run 3 iterations
        df_next_th = deduplicate(df.copy(deep=True), th=th, min_threshold=0.3, num_best=10)
        
        # run this block the second time, to identify and fix clsuters with low cluster_score
        if "cluster_number" in df.columns.tolist():
            # identify clsuters with low cluster_score
            potentially_incorrect_clusters = df[df.cluster_score <= TH_CLUSTER_BREAK].cluster_number.unique()
            mask = df.cluster_number.isin(potentially_incorrect_clusters)
            df_bad = df[mask]
            print("Number of potentially incorrect clusters: %d" % len(potentially_incorrect_clusters))
            
            # Replace bad clusters with new clusters run with a higher threshold
            df_bad_fixed = df_next_th[df_next_th.TID.isin(df_bad.TID)].set_index("TID", drop=False)
            assert df_bad_fixed.shape[0] == df_bad.shape[0]
            for col in ["cluster_number", "cluster_score", "lowest_score", "cluster_size"]:
                # We use a "hack" here to make sure the new cluster numbers are not overlapping with old ones
                # Mutiplying MAX_NUM*th should generate a new set of cluster numbers 
                # as long as MAX_NUM * 0.1 > len(df)
                df.loc[df_bad_fixed.TID, col] = df_bad_fixed[col] + (MAX_NUM*th if col == 'cluster_number' else 0)
                
        else: # first time; prepare for the next run
            df = df_next_th

        out = compute_precision_recall(df)
        print("Ran for threshold: %.2f" %(th))
        print(out)
    return out


result = run(0.45)

2022-01-20 20:21:09,419 : INFO : Ingesting data into an OOC embedded database
2022-01-20 20:21:09,421 : INFO : Phone number columns: []
2022-01-20 20:21:09,424 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:21:09,429 : INFO : Address columns: []


Shape: 19375 X 13
Number of clusters: 10000


2022-01-20 20:21:10,113 : INFO : Blocking 19375 records on columns ['title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:21:10,211 : INFO : Using out-of-core tokenizer 
2022-01-20 20:21:16,721 : INFO : TFIDF blocker created with 31991 tokens
2022-01-20 20:21:16,723 : INFO : Saving EntityFinder blocked model to /build/paper_benchmarks/music
2022-01-20 20:21:16,923 : INFO : Loading using memorymap: r
2022-01-20 20:21:17,023 : INFO : Set dummy = -1
2022-01-20 20:21:17,024 : INFO : Collecting dictionary stat
2022-01-20 20:21:17,041 : INFO : Actually creating the index
2022-01-20 20:21:17,078 : INFO : Sanity check
2022-01-20 20:21:17,092 : INFO : Phone number columns: []
2022-01-20 20:21:17,093 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:21:17,093 : INFO : Address columns: []
2022-01-20 20:21:17,383 : INFO : Searching through blocked/indexed data to pick top 10 matches
2022-01-20 20:21:28,973

Ran for threshold: 0.45
{'precision': 0.8758509990590579, 'recall': 0.9737846153846154, 'f_measure': 0.9222251362298568, 'num_clusters': 9828, 'precision_identified_links': 18067.0, 'precision_correctly_identified_links': 15824.0, 'recall_num_links': 16250.0, 'recall_correctly_identified_links': 15824.0}


2022-01-20 20:22:08,559 : INFO : Blocking 19375 records on columns ['title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:22:08,614 : INFO : Using out-of-core tokenizer 
2022-01-20 20:22:13,872 : INFO : TFIDF blocker created with 31991 tokens
2022-01-20 20:22:13,874 : INFO : Saving EntityFinder blocked model to /build/paper_benchmarks/music
2022-01-20 20:22:14,084 : INFO : Loading using memorymap: r
2022-01-20 20:22:14,182 : INFO : Set dummy = -1
2022-01-20 20:22:14,184 : INFO : Collecting dictionary stat
2022-01-20 20:22:14,201 : INFO : Actually creating the index
2022-01-20 20:22:14,239 : INFO : Sanity check
2022-01-20 20:22:14,252 : INFO : Phone number columns: []
2022-01-20 20:22:14,253 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:22:14,254 : INFO : Address columns: []
2022-01-20 20:22:14,550 : INFO : Searching through blocked/indexed data to pick top 10 matches
2022-01-20 20:22:26,141

...

Number of potentially incorrect clusters: 70


2022-01-20 20:24:08,850 : INFO : Ingesting data into an OOC embedded database
2022-01-20 20:24:08,852 : INFO : Phone number columns: []
2022-01-20 20:24:08,853 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:24:08,854 : INFO : Address columns: []


Ran for threshold: 0.65
{'precision': 0.961147601933325, 'recall': 0.9545230769230769, 'f_measure': 0.9578238853896505, 'num_clusters': 10149, 'precision_identified_links': 16138.0, 'precision_correctly_identified_links': 15511.0, 'recall_num_links': 16250.0, 'recall_correctly_identified_links': 15511.0}


2022-01-20 20:24:09,633 : INFO : Blocking 19375 records on columns ['title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:24:09,720 : INFO : Using out-of-core tokenizer 
2022-01-20 20:24:17,974 : INFO : TFIDF blocker created with 31991 tokens
2022-01-20 20:24:17,976 : INFO : Saving EntityFinder blocked model to /build/paper_benchmarks/music
2022-01-20 20:24:18,124 : INFO : Loading using memorymap: r
2022-01-20 20:24:18,180 : INFO : Set dummy = -1
2022-01-20 20:24:18,180 : INFO : Collecting dictionary stat
2022-01-20 20:24:18,194 : INFO : Actually creating the index
2022-01-20 20:24:18,218 : INFO : Sanity check
2022-01-20 20:24:18,228 : INFO : Phone number columns: []
2022-01-20 20:24:18,228 : INFO : String columns: ['TID', 'title', 'artist', 'album', 'language', 'number', 'length', 'year']
2022-01-20 20:24:18,229 : INFO : Address columns: []
2022-01-20 20:24:18,446 : INFO : Searching through blocked/indexed data to pick top 10 matches
2022-01-20 20:24:32,354

Number of potentially incorrect clusters: 16
Ran for threshold: 0.75
{'precision': 0.9643435980551054, 'recall': 0.952, 'f_measure': 0.9581320450885668, 'num_clusters': 10169, 'precision_identified_links': 16042.0, 'precision_correctly_identified_links': 15470.0, 'recall_num_links': 16250.0, 'recall_correctly_identified_links': 15470.0}


### Final result: 

In [7]:
result

{'precision': 0.9643435980551054,
 'recall': 0.952,
 'f_measure': 0.9581320450885668,
 'num_clusters': 10169,
 'precision_identified_links': 16042.0,
 'precision_correctly_identified_links': 15470.0,
 'recall_num_links': 16250.0,
 'recall_correctly_identified_links': 15470.0}